In [1]:
# Importing libraries

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [2]:
pip install torch

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
movies=pd.read_csv(r'C:\Users\Krishi Thiruppathi\Desktop\ml-1m\movies.dat',sep='::', header=None, engine='python', encoding='latin-1')

In [4]:
movies.head(10)

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [5]:
users=pd.read_csv(r'C:\Users\Krishi Thiruppathi\Desktop\ml-1m\users.dat',sep='::', header=None, engine='python', encoding='latin-1')

In [6]:
users.head(10)

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455
5,6,F,50,9,55117
6,7,M,35,1,06810
7,8,M,25,12,11413
8,9,M,25,17,61614
9,10,F,35,1,95370


In [7]:
ratings=pd.read_csv(r'C:\Users\Krishi Thiruppathi\Desktop\ml-1m\ratings.dat',sep='::', header=None, engine='python', encoding='latin-1')

In [8]:
ratings.head(10)

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
5,1,1197,3,978302268
6,1,1287,5,978302039
7,1,2804,5,978300719
8,1,594,4,978302268
9,1,919,4,978301368


In [9]:
# Preparing training set
training_set = pd.read_csv(r'C:\Users\Krishi Thiruppathi\Desktop\ml-100k\u1.base',delimiter='\t')

# Converting to arrays - to use pytorch tensors
training_set=np.array(training_set)
training_set

array([[        1,         2,         3, 876893171],
       [        1,         3,         4, 878542960],
       [        1,         4,         3, 876893119],
       ...,
       [      943,      1188,         3, 888640250],
       [      943,      1228,         3, 888640275],
       [      943,      1330,         3, 888692465]], dtype=int64)

In [10]:
# Preparing test set
test_set = pd.read_csv(r'C:\Users\Krishi Thiruppathi\Desktop\ml-100k\u1.test',delimiter='\t')

# Converting to arrays - to use pytorch tensors
test_set=np.array(test_set)
test_set

array([[        1,        10,         3, 875693118],
       [        1,        12,         5, 878542960],
       [        1,        14,         5, 874965706],
       ...,
       [      459,       934,         3, 879563639],
       [      460,        10,         3, 882912371],
       [      462,       682,         5, 886365231]], dtype=int64)

In [11]:
# Getting the number of users and movies

nb_users=int(max(max(training_set[:,0]) , max(test_set[:,0])))
nb_movies=int(max(max(training_set[:,1]) , max(test_set[:,1])))

In [12]:
print("Total users:",nb_users,',',"Total no. of movies:",nb_movies)

Total users: 943 , Total no. of movies: 1682


In [13]:
# Converting the data into an array with users in lines and movies in columns
# Providing 943 lists(each user 1 list) for each user to rate for 1682 movies. If no rating is provided, we substitute a 0.

def convert(data):
    new_data = []
    for id_users in range(1,nb_users+1):
        id_movies = data[:,1][data[:,0]=='id_users']
        id_ratings = data[:,2][data[:,0]=='id_users']
        ratings = np.zeros(nb_movies)
        ratings[id_movies-1] = id_ratings
        new_data.append(list[ratings])
    return new_data

training_set=convert(training_set)
test_set=convert(test_set)

C:\Users\Krishi Thiruppathi\AppData\Local\Temp\ipykernel_5716\1217804643.py:7: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  id_movies = data[:,1][data[:,0]=='id_users']
C:\Users\Krishi Thiruppathi\AppData\Local\Temp\ipykernel_5716\1217804643.py:8: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  id_ratings = data[:,2][data[:,0]=='id_users']


In [14]:
training_set

[list[array([0., 0., 0., ..., 0., 0., 0.])],
 list[array([0., 0., 0., ..., 0., 0., 0.])],
 list[array([0., 0., 0., ..., 0., 0., 0.])],
 list[array([0., 0., 0., ..., 0., 0., 0.])],
 list[array([0., 0., 0., ..., 0., 0., 0.])],
 list[array([0., 0., 0., ..., 0., 0., 0.])],
 list[array([0., 0., 0., ..., 0., 0., 0.])],
 list[array([0., 0., 0., ..., 0., 0., 0.])],
 list[array([0., 0., 0., ..., 0., 0., 0.])],
 list[array([0., 0., 0., ..., 0., 0., 0.])],
 list[array([0., 0., 0., ..., 0., 0., 0.])],
 list[array([0., 0., 0., ..., 0., 0., 0.])],
 list[array([0., 0., 0., ..., 0., 0., 0.])],
 list[array([0., 0., 0., ..., 0., 0., 0.])],
 list[array([0., 0., 0., ..., 0., 0., 0.])],
 list[array([0., 0., 0., ..., 0., 0., 0.])],
 list[array([0., 0., 0., ..., 0., 0., 0.])],
 list[array([0., 0., 0., ..., 0., 0., 0.])],
 list[array([0., 0., 0., ..., 0., 0., 0.])],
 list[array([0., 0., 0., ..., 0., 0., 0.])],
 list[array([0., 0., 0., ..., 0., 0., 0.])],
 list[array([0., 0., 0., ..., 0., 0., 0.])],
 list[arra

In [22]:
# Converting data into torch tensors
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)


In [23]:
# Converting all ratings into binary ratings(0-not liked or 1-liked)- to be inputs of Restricted Boltzmann Machines(RBM)

# Ratings not given by users shd be given another value(we assigned 0 but now since only binary inputs, we allocate -1 to them)
training_set[training_set==0]=-1
# 1 and 2 star ratings - 0
training_set[training_set==1]=0
training_set[training_set==2]=0
# 3,4 and 5 star ratings - 1
training_set[training_set>=3]=1

# For test set
test_set[test_set==0]=-1
# 1 and 2 star ratings - 0
test_set[test_set==1]=0
test_set[test_set==2]=0
# 3,4 and 5 star ratings - 1
test_set[test_set>=3]=1

In [47]:
# Creating architecture of neural networks-RBM
class RBM():
     # nv-no. of visible nodes, nh-no. of hidden nodes
    def __init__(self, nv, nh): # Initialise the weights and bias(parameters of future objects)
        self.W = torch.randn(nh, nv)
        self.a = torch.randn(1, nh) # a-bias for hidden node wrt visible node(1- creates a 2D tensor)
        self.b = torch.randn(1, nv) # b-bias for visible node wrt hidden node
        
    def sample_h(self, x): # Calculate the probability a hidden node is 1 given the values of the visible nodes i.e sigmoid activation function
        wx = torch.mm(x, self.W.t())
        activation = wx + self.a.expand_as(wx) #wx+a
        p_h_given_v = torch.sigmoid(activation)
        return p_h_given_v, torch.bernoulli(p_h_given_v) # probabilities, 1s and 0s.
    
    def sample_v(self, y): # Calculate the probability a visible node is 1 given the values of the hidden nodes
        wy = torch.mm(y, self.W)
        activation = wy + self.b.expand_as(wy)  # wy+a, using expand_as function to apply bias to each line of mini-batch
        p_v_given_h = torch.sigmoid(activation)
        return p_v_given_h, torch.bernoulli(p_v_given_h)
    
    def train(self, v0, vk, ph0, phk): # v0- input vector, vk-visible nodes obtained after k samplings
        # formulae involved in k-step Contrastive divergence
        self.W += (torch.mm(v0.t(),ph0) - torch.mm(vk.t(),phk)).t()
        self.b += torch.sum((v0 - vk), 0) # to keep format of b as a tensor of 2-D
        self.a += torch.sum((ph0 - phk), 0)

In [48]:
training_set[0]

tensor([-1.,  1.,  1.,  ..., -1., -1., -1.])

In [49]:
# Creating the RBM object

nv = len(training_set[0])
nh = 100 # starting with 100 features
batch_size = 100
rbm = RBM(nv,nh)

In [50]:
# Training the RBM

nb_epoch = 10
for epoch in range(1, nb_epoch+1):
    train_loss = 0
    s = 0.
    for id_user in range(0, nb_users-batch_size, batch_size): # no. of batches: 0-99,100-199....843-943
        
        # at beginning, input(vk) is same as target(v0) and later will be updated
        vk = training_set[id_user:id_user + batch_size] # vk-Input(batch of observations), will be the output of Gibbs sampling after k-steps of random walk
        v0 = training_set[id_user:id_user + batch_size] # v0-ratings of the movie already rated by the 100 users in the batch
        ph0,_ = rbm.sample_h(v0)
        
        # for loop for k-steps of contrastive divergence(Gibbs chain)
        for k in range(10):
            _,hk = rbm.sample_h(vk)
            _,vk = rbm.sample_v(hk
            vk[v0<0] = v0[v0<0]
        phk,_ = rbm.sample_h(vk)
        rbm.train(v0, vk, ph0, phk)
        
        train_loss += torch.mean(torch.abs(v0[v0>=0] - vk[v0>=0]))
        s += 1.
    print('Epoch:'+ str(epoch)+'Loss:'+str(train_loss/s))
            

Epoch:1Loss:tensor(0.3625)
Epoch:2Loss:tensor(0.2437)
Epoch:3Loss:tensor(0.2532)
Epoch:4Loss:tensor(0.2417)
Epoch:5Loss:tensor(0.2482)
Epoch:6Loss:tensor(0.2468)
Epoch:7Loss:tensor(0.2455)
Epoch:8Loss:tensor(0.2471)
Epoch:9Loss:tensor(0.2487)
Epoch:10Loss:tensor(0.2479)


In [51]:
# Testing the RBM

test_loss = 0
s = 0.
for id_user in range(nb_users): 

    # at beginning, input(vk) is same as target(v0) and later will be updated
    v = training_set[id_user:id_user+1] # training_set is the input used to activate the hidden neurons to get the output
    vt = test_set[id_user:id_user+1] #vt-target

    # For 1-step of contrastive divergence(Gibbs chain)
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
        s += 1.
print('Test Loss:'+str(test_loss/s))


Test Loss:tensor(0.2452)
